<a href="https://colab.research.google.com/github/DataScienceLab-DSLABI/Active-Learning-Anomaly-Detection/blob/main/%5BDS_Lab_I%5D_First_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/icsdataset/hai
! gunzip *.gz
! git lfs pull

Cloning into 'hai'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 329 (delta 37), reused 66 (delta 15), pack-reused 222
Receiving objects: 100% (329/329), 519.79 MiB | 31.69 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Updating files: 100% (72/72), done.
Filtering content: 100% (20/20), 1.65 GiB | 48.50 MiB/s, done.
gzip: *.gz: No such file or directory
Not in a git repository.


In [3]:
print('test to change')

test to change
